# BYOBU

In [1]:
from models import DINOv2ViT, CustomResNet
import torch
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

/storage/disk0/arda/dinov2/distillation/../../dinov2/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/storage/disk0/arda/dinov2/distillation/../../dinov2/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/storage/disk0/arda/dinov2/distillation/../../dinov2/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


In [2]:
import torch
x = torch.randn(1, 3, 224, 224).to(device)

teacher = DINOv2ViT().to(device)


Using cache found in /home/arda/.cache/torch/hub/facebookresearch_dinov2_main


In [3]:
from torch import nn

out = teacher(x)
print(out["patch_embeddings"].shape)
print(out["embedding"].shape)
print(out["feature_map"].shape)

# tests/test_resnet.py
import torch
from models.resnet import ResNet, BasicStem, BottleneckBlock, make_resnet_stages

import torch
from models.student_model_wrapper import ModelWrapper

# Define the device
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# Sample input
x = torch.randn(1, 3, 224, 224).to(device)

# Feature matcher configuration (customize as needed)
feature_matcher_config = {
    'in_channels': 2048,
    'out_channels': 1536,
    'kernel_size': 1,
    'stride': 1,
    'padding': 0,
    'activation': 'ReLU'  # Options: 'ReLU', 'LeakyReLU', etc.
}

# Initialize the ModelWrapper with desired parameters
model_wrapper = ModelWrapper(
    model_type='resnet',
    model_depth=50,
    block_class=BottleneckBlock,
    norm_type='BN',
    in_channels=3,
    feature_matcher_config=feature_matcher_config,
    device=device
)

# Forward pass through the wrapper
matched_features, student_output = model_wrapper(x)

# Display the shape of the matched features
if matched_features is not None:
    print(f"Matched Features Shape: {matched_features.shape}")
else:
    print("Feature Matcher is not configured.")

# Optionally, get the shape of 'res5' feature map
res5_shape = model_wrapper.get_feature_map_shape(x, feature_key='res5')
print(f"Shape of 'res5' feature map: {res5_shape}")


/home/arda/miniconda3/envs/dinov2/lib/python3.9/site-packages/xformers/ops/unbind.py:46: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage_data_ptr = tensors[0].storage().data_ptr()
/home/arda/miniconda3/envs/dinov2/lib/python3.9/site-packages/xformers/ops/unbind.py:48: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if x.storage().data_ptr() != storage_data_ptr:


torch.Size([1, 256, 1536])
torch.Size([1, 1536])
torch.Size([1, 1536, 16, 16])
Matched Features Shape: torch.Size([1, 1536, 16, 16])
Shape of 'res5' feature map: torch.Size([1, 2048, 7, 7])


In [10]:
model_wrapper.state_dict().keys()

odict_keys(['student.stem.conv1.weight', 'student.stem.conv1.norm.weight', 'student.stem.conv1.norm.bias', 'student.stem.conv1.norm.running_mean', 'student.stem.conv1.norm.running_var', 'student.stem.conv1.norm.num_batches_tracked', 'student.res2.0.shortcut.weight', 'student.res2.0.shortcut.norm.weight', 'student.res2.0.shortcut.norm.bias', 'student.res2.0.shortcut.norm.running_mean', 'student.res2.0.shortcut.norm.running_var', 'student.res2.0.shortcut.norm.num_batches_tracked', 'student.res2.0.conv1.weight', 'student.res2.0.conv1.norm.weight', 'student.res2.0.conv1.norm.bias', 'student.res2.0.conv1.norm.running_mean', 'student.res2.0.conv1.norm.running_var', 'student.res2.0.conv1.norm.num_batches_tracked', 'student.res2.0.conv2.weight', 'student.res2.0.conv2.norm.weight', 'student.res2.0.conv2.norm.bias', 'student.res2.0.conv2.norm.running_mean', 'student.res2.0.conv2.norm.running_var', 'student.res2.0.conv2.norm.num_batches_tracked', 'student.res2.0.conv3.weight', 'student.res2.0.con

In [5]:
# from datasets.GTA5 import GTA5Dataset
import sys
sys.path.append('./datasets')  # Add the datasets directory to the Python path
from GTA5 import GTA5Dataset
from collate_fn import collate_data_and_cast
from dinov2.data.augmentations import DataAugmentationDINO
from torch.utils.data import DataLoader


import yaml

# Load configurations
with open("config/config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

# Data Transformation
data_transform = DataAugmentationDINO(
    global_crops_scale=tuple(cfg['data_transform']['global_crops_scale']),
    local_crops_scale=tuple(cfg['data_transform']['local_crops_scale']),
    local_crops_number=cfg['data_transform']['n_local_crops'],
    global_crops_size=tuple(cfg['data_transform']['global_crops_size']),
    local_crops_size=tuple(cfg['data_transform']['local_crops_size']),
)

# Dataset and DataLoader
dataset = GTA5Dataset(img_dir = '/home/arda/data/train2017', transform=data_transform)
# data_loader = DataLoader(
#     dataset,
#     batch_size=cfg['data_loader']['batch_size'],
#     num_workers=cfg['data_loader']['num_workers'],
#     shuffle=cfg['data_loader']['shuffle'],
#     collate_fn=collate_data_and_cast
# )

# Split dataset into train and test sets
from torch.utils.data import random_split, Subset
import random

# Sample a smaller subset of the data
total_size = len(dataset)

# Split sampled dataset into train/test
train_size = int(0.9 * total_size)  # 90% for training
test_size = total_size - train_size  # 10% for testing

# Create train and test datasets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create train and test dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=cfg['data_loader']['batch_size'], 
    num_workers=cfg['data_loader']['num_workers'],
    shuffle=cfg['data_loader']['shuffle'],
    collate_fn=collate_data_and_cast
)

test_loader = DataLoader(
    test_dataset,
    batch_size=cfg['data_loader']['batch_size'],
    num_workers=cfg['data_loader']['num_workers'], 
    shuffle=False,
    collate_fn=collate_data_and_cast
)

# Optimizer
optimizer = getattr(torch.optim, cfg['optimizer']['type'])([
    {"params": model_wrapper.parameters()},
], lr=1e-3)

# Freeze teacher model
for param in teacher.parameters():
    param.requires_grad = False


In [ ]:
import torch
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import os
import logging
from pytorch_metric_learning import miners, losses
import torch.optim.lr_scheduler as lr_scheduler  # Added for scheduler

miner = miners.BatchHardMiner()
contrastive_loss = losses.ContrastiveLoss()
logging.basicConfig(filename='training.log', level=logging.INFO)


best_test_similarity = 0
save_frequency = 5
checkpoint_dir = "./checkpoints"
scaler = GradScaler()  # For mixed precision training

def compute_feature_similarity(feat1, feat2):
    f1 = feat1.reshape(-1, feat1.shape[-1])
    f2 = feat2.reshape(-1, feat2.shape[-1])
    similarity = F.cosine_similarity(f1, f2, dim=1)
    return similarity.mean()

def compound_loss(mse_loss, cosine_sim_loss, alpha=1.0, beta=1.0):
    return alpha * mse_loss + beta * cosine_sim_loss

checkpoint_path = os.path.join(checkpoint_dir, "latest_checkpoint.pth")

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model_wrapper.student.load_state_dict(checkpoint['student_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_iteration = checkpoint['iteration'] + 1
    best_test_similarity = checkpoint['best_test_similarity']
    print(f"Resuming from iteration {start_iteration}")

total_iterations = 30 * len(train_loader)  # Example: 30 epochs worth of iterations

# Initialize the Cosine Annealing Scheduler
total_epochs = 30  # Adjust this based on your actual number of epochs
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_epochs, eta_min=1e-6)

data_iterator = iter(tqdm(train_loader, desc="Preparing data iterator"))

for iteration in range(start_iteration if 'start_iteration' in locals() else 0, total_iterations):
    try:
        data = next(data_iterator)
    except StopIteration:
        data_iterator = iter(tqdm(train_loader, desc="Restarting data iterator"))
        data = next(data_iterator)
    
    if iteration % len(train_loader) == 0:
        model_wrapper.train()
        teacher.eval()
        epoch = iteration // len(train_loader)
        epoch_loss = []
        similarities = []
        embedding_losses = []

    global_crops = data["collated_global_crops"].to(device)
    # local_crops = data["collated_local_crops"].to(device)
    batch_size = int(global_crops.shape[0]//2)
    labels = torch.arange(batch_size).repeat(2)  # Creates [0,1,2,...,batch_size-1, 0,1,2,...,batch_size-1]
    labels = labels.to(device)
    
    with torch.no_grad():
        teacher_output = teacher(global_crops)
        teacher_feature_maps = teacher_output["feature_map"]

    with autocast():  # Mixed precision
        student_output = model_wrapper(global_crops)
        student_feature_maps = student_output["dinov2_feature_map"]
        # student_contrastive_embeddings = student_output["contrastive_embeddings"]
        mse_loss = F.mse_loss(student_feature_maps, teacher_feature_maps)
        student_feature_normalized = F.normalize(student_feature_maps, p=2, dim=1)
        teacher_feature_normalized = F.normalize(teacher_feature_maps, p=2, dim=1)
        cosine_similarity = F.cosine_similarity(student_feature_normalized, teacher_feature_normalized, dim=1)
        cosine_similarity_loss = 1 - cosine_similarity.mean()
        # embedding_loss = contrastive_loss(student_contrastive_embeddings, labels)
        # embedding_losses.append(embedding_loss.item())
        total_loss = compound_loss(mse_loss, cosine_similarity_loss, alpha=1.0, beta=0) # + embedding_loss

    scaler.scale(total_loss).backward()
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad()

    similarity = compute_feature_similarity(student_feature_maps, teacher_feature_maps)
    similarities.append(similarity.item())
    epoch_loss.append(total_loss.item())

    if (iteration + 1) % len(train_loader) == 0:
        # Evaluation at the end of each epoch-equivalent
        model_wrapper.eval()
        test_losses = []
        test_similarities = []
        with torch.no_grad():
            for i, data in enumerate(tqdm(test_loader, desc="Testing")):
                global_crops = data["collated_global_crops"].to(device)
                teacher_output = teacher(global_crops)
                student_output = model_wrapper(global_crops)

                test_mse = F.mse_loss(student_output["dinov2_feature_map"], teacher_output["feature_map"])
                test_similarity = compute_feature_similarity(student_output["dinov2_feature_map"], teacher_output["feature_map"])

                test_losses.append(test_mse.item())
                test_similarities.append(test_similarity.item())

        avg_train_loss = sum(epoch_loss) / len(epoch_loss)
        avg_train_similarity = sum(similarities) / len(similarities)
        avg_test_loss = sum(test_losses) / len(test_losses)
        avg_test_similarity = sum(test_similarities) / len(test_similarities)
        current_epoch = (iteration + 1) // len(train_loader)
        print(f"Epoch {current_epoch}")
        print(f"Train Loss: {avg_train_loss:.4f}")
        print(f"Train Similarity: {avg_train_similarity:.4f}")
        print(f"Test Loss: {avg_test_loss:.4f}")
        print(f"Test Similarity: {avg_test_similarity:.4f}")
        # print(f"Embedding Loss: {avg_embedding_loss:.4f}")

        logging.info(f"Epoch {current_epoch}")
        logging.info(f"Train Loss: {avg_train_loss:.4f}")
        logging.info(f"Train Similarity: {avg_train_similarity:.4f}")
        logging.info(f"Test Loss: {avg_test_loss:.4f}")
        logging.info(f"Test Similarity: {avg_test_similarity:.4f}")
        
        if (current_epoch) % save_frequency == 0:
            checkpoint = {
                'iteration': iteration,
                'student_state_dict': model_wrapper.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': avg_train_loss,
                'test_loss': avg_test_loss,
                'train_similarity': avg_train_similarity,
                'test_similarity': avg_test_similarity,
                'best_test_similarity': best_test_similarity
            }
            torch.save(checkpoint, os.path.join(checkpoint_dir, f"checkpoint_iter_{iteration}.pth"))
            torch.save(checkpoint, os.path.join(checkpoint_dir, "latest_checkpoint.pth"))

        if avg_test_similarity > best_test_similarity:
            best_test_similarity = avg_test_similarity
            torch.save({
                'iteration': iteration,
                'student_state_dict': model_wrapper.state_dict(),
                'test_similarity': avg_test_similarity
            }, os.path.join(checkpoint_dir, "best_model.pth"))
        
        # Step the scheduler at the end of the epoch
        scheduler.step()
        print(f"Learning Rate adjusted to: {scheduler.get_last_lr()[0]:.6f}")

Preparing data iterator:   0%|                                                                                                                              | 0/832 [00:00<?, ?it/s]/home/arda/miniconda3/envs/dinov2/lib/python3.9/site-packages/xformers/ops/unbind.py:46: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage_data_ptr = tensors[0].storage().data_ptr()
/home/arda/miniconda3/envs/dinov2/lib/python3.9/site-packages/xformers/ops/unbind.py:48: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if x.storage().da

# # TODO

1 - Match the feature map sizes, then apply dino and ibot losses

In [ ]:
# Save only the backbone weights of student model
torch.save({
    'backbone_state_dict': student.state_dict(),
}, 'student_backbone_checkpoint.pth')

print("Student backbone weights saved successfully")



In [ ]:
import torch    
vitb8 = torch.hub.load('facebookresearch/dino:main', 'dino_vitb8')

# Load the checkpoint
checkpoint = torch.load('../../checkpoints/dino_vitbase8_pretrain_full_checkpoint.pth')

# Check if the checkpoint contains the state_dict for the model
if 'state_dict' in checkpoint:
    state_dict = checkpoint['state_dict']
else:
    state_dict = checkpoint  # If the checkpoint is directly the state_dict

# Load the state_dict into the model
vitb8.load_state_dict(state_dict, strict=False)

# Print a message to confirm successful loading
print("Weights loaded successfully into the DINO model.")

In [18]:
checkpoint['student'].keys()





dict_keys(['module.backbone.cls_token', 'module.backbone.pos_embed', 'module.backbone.patch_embed.proj.weight', 'module.backbone.patch_embed.proj.bias', 'module.backbone.blocks.0.norm1.weight', 'module.backbone.blocks.0.norm1.bias', 'module.backbone.blocks.0.attn.qkv.weight', 'module.backbone.blocks.0.attn.qkv.bias', 'module.backbone.blocks.0.attn.proj.weight', 'module.backbone.blocks.0.attn.proj.bias', 'module.backbone.blocks.0.norm2.weight', 'module.backbone.blocks.0.norm2.bias', 'module.backbone.blocks.0.mlp.fc1.weight', 'module.backbone.blocks.0.mlp.fc1.bias', 'module.backbone.blocks.0.mlp.fc2.weight', 'module.backbone.blocks.0.mlp.fc2.bias', 'module.backbone.blocks.1.norm1.weight', 'module.backbone.blocks.1.norm1.bias', 'module.backbone.blocks.1.attn.qkv.weight', 'module.backbone.blocks.1.attn.qkv.bias', 'module.backbone.blocks.1.attn.proj.weight', 'module.backbone.blocks.1.attn.proj.bias', 'module.backbone.blocks.1.norm2.weight', 'module.backbone.blocks.1.norm2.bias', 'module.bac

In [22]:
checkpoint['student']['module.head.last_layer.weight'].shape


torch.Size([65536, 256])

: 